In [2]:
# imports
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.colors import Normalize
from ies_model_library import *
from iessolution import *
import scipy.stats
import diptest
import matplotlib.ticker as mtick
from linear_regression import read_raw_data
import pyomo.environ as pyo


ModuleNotFoundError: No module named 'ies_model_library'

In [ ]:

def read_all_files(directory):
    '''
    Loops through a given directory and converts all the files in the directory to solution objects 

    Arguments: 
        directory: string, where the files are located 

    Returns:
        solution_objects: list of IESSolution classes, classes represent each solution
    '''
    # make a list of signal names
    label_list = os.listdir(directory)

    # remove 00time item
    label_list.remove('00_time.csv')
    label_list.remove('.ipynb_checkpoints')

    # remove repeats
    remove_list = []
    for s in label_list:
        if 'ERCOT_0' in s or 'ERCOT_100' in s or 'CAISO_100' in s:
            remove_list.append(s)

    for r in remove_list:
        label_list.remove(r)


    # read the lmp statistics info
    formatted_data = read_raw_data('formatted_raw_data.csv')


    # make a list of ies_model objects
    iesmodels = {'_model0':Case0(), '_model1':Case1(), '_model3':Case3(), '_model4':Case4(), '_model5':Case5(), '_model6':Case6()}

    solution_objects = []

    # define label in solution object 
    sub = '.csv'

    # loop through the label list
    for i in range(len(label_list)):

        # read csv 
        results_csv = pd.read_csv(directory + '/' + label_list[i])

        case_name = label_list[i][:-len(sub)]

        # make solution object 
        soln = IESSolution(csv_file = directory + '/' + label_list[i], case_object = iesmodels[case_name[len(case_name)-7:]])

        # add a gas price to the solution object
        soln.gas_price = formatted_data.at[(label_list[i][:-(len(sub)+7)] + directory[-3:]), 'Natural Gas Price ($/MMBtu)']

        # make a list of soln objects
        solution_objects.append(soln)

    
    # save a signal label and natural gas price for each
    for i in range(len(solution_objects)):

        solution_objects[i].signal_label = label_list[i][:-len(sub)]

    return solution_objects

def separate_cases(solution_objects):
    '''
    Separates a list of solution objects into 6 individual case lists 
    This function assumes file contains cases 0,1,3,4,5,6 and that the files are sorted by lmp_signal_name_caseX

    Arguments:
        solution_objects: list of IESSolution classes, contains all cases

    Returns:
        case0_objects: list of IESSolution classes, only case 0 
        case1_objects: list of IESSolution classes, only case 1  
        case3_objects: list of IESSolution classes, only case 3 
        case4_objects: list of IESSolution classes, only case 4
        case5_objects: list of IESSolution classes, only case 5 
        case6_objects: list of IESSolution classes, only case 6 
    '''

    # initialize empty lists
    case0_objects = []
    case1_objects = []
    case3_objects = []
    case4_objects = []
    case5_objects = []
    case6_objects = []
        
    for i in range(len(solution_objects)):

        # return the model using the end of the string
        length = len(solution_objects[i].signal_label)

        # trim the last 7 characters '_modelX'

        model_num = solution_objects[i].signal_label[length - 7:]
        # print(model_num)

        
        # append lists based on model number 
        if model_num == '_model0':
            case0_objects.append(solution_objects[i])
        elif model_num == '_model1':
            case1_objects.append(solution_objects[i])
        elif model_num == '_model3':
            case3_objects.append(solution_objects[i])
        elif model_num == '_model4':
            case4_objects.append(solution_objects[i])
        elif model_num == '_model5':
            case5_objects.append(solution_objects[i])
        elif model_num == '_model6':
            case6_objects.append(solution_objects[i])
        else:
            raise NotImplementedError()
        
    # check if model 0 and model 1 are empty 
    if not case0_objects and not case1_objects:
        # read the market results 20 file 
        objects2 = read_all_files('../market_results_20')

        for i in range(len(objects2)):

            # return the model using the end of the string
            length = len(objects2[i].signal_label)

            model_num = objects2[i].signal_label[length - 7:]

            # append 0 and 1 lists only
            if model_num == '_model0':
                case0_objects.append(objects2[i])
            elif model_num == '_model1':
                case1_objects.append(objects2[i])


    
    return case0_objects, case1_objects, case3_objects, case4_objects, case5_objects, case6_objects


In [ ]:
def plt_signal_comparison(directory, PROFIT = True, CAPACITY = True, OUTPUT = True, sort = 'median', relative_case = None, save = False):
    '''
    Reads all the result .csv files in given directory and plots each's profit, capacity factor and/or plant output. 
    Saves files in the directory

    Arguments:
        directory: string, directory where the csv files are 
        PROFIT: boolean, plots profit 
        CAPACITY: boolean, plots capacity factor
        OUTPUT: boolean, plots plant output
        sort: string, what to sort the lmp signals by 
            options: median (default): sorted by lmp signal median value
            bimodal: sorted by value of bimodality coefficient 
            gasprice: sorted by natural gas price of the signal
            source: sorted by signal source, chronologically
        relative_case: integer, case number to reference other values to in plot 
            default: None, no relative case, all numbers are plotted in their original scale
        save: boolean, whether or not to save copies of the plots 
            default = False, do not save

    Returns:
        none 
    '''

    solution_objects = read_all_files(directory)

    # plotting data 
    plot_data = {0 : {}, 1 : {}, 3 : {}, 4 : {}, 5: {}, 6: {}}

    # generate keys based on options selected 
    for key in plot_data.keys():
        # case_objects category 
        plot_data[key]['case objects'] = []

        # profit
        if PROFIT:
            plot_data[key]['profit'] = []

        # capacity factor
        if CAPACITY:
            plot_data[key]['p capacity'] = []
            plot_data[key]['h capacity'] = []

        # output
        if OUTPUT:
            plot_data[key]['p output'] = []
            plot_data[key]['h output'] = []


    # sort the objects chosen sorting method 
    if sort == 'median':
        sort_func = lambda soln_obj: np.median(soln_obj.lmp)
    elif sort == 'bimodal':
        sort_func = lambda soln_obj: ((scipy.stats.skew(soln_obj.lmp)**2)+1)/(scipy.stats.kurtosis(soln_obj.lmp) + ((3*(len(soln_obj.lmp)-1)**2)/((len(soln_obj.lmp)-2)*(len(soln_obj.lmp)-3))))
    elif sort == 'gasprice':
        sort_func = lambda soln_obj: soln_obj.gas_price
    elif sort == 'source':
        sort_func = None
    
    else:
        raise NotImplementedError('Please choose to sort lmp signals by median of bimodality coefficient by setting sort to median or bimodal')

    # use python sort function
    if sort != 'source':
        solution_objects.sort(key = sort_func)
    else:
        # perform sort by source 

        # separate list into years and different projections 
        list_2019 = []
        list_2022 = []
        list_nrel = []
        list_princeton = []
        list_netl = []

        for obj in solution_objects:
            # print(obj.signal_label[-11:-8])
            if obj.signal_label[-11:-7] == '2019':
                list_2019.append(obj)
            elif obj.signal_label[-11:-7] == '2022':
                list_2022.append(obj)

            elif obj.signal_label[-11:-7] == '2030':
                list_princeton.append(obj)
            elif obj.signal_label[-11:-7] == '2035':
                # check nrel 
                if obj.signal_label[0:4] == 'MiNg':
                    list_nrel.append(obj)
            else:
                list_netl.append(obj)

        # sort each of these lists by median lmp 
        sort_func = lambda soln_obj: np.median(soln_obj.lmp)

        list_2019.sort(key = sort_func)
        list_2022.sort(key = sort_func)
        list_nrel.sort(key = sort_func)
        list_princeton.sort(key = sort_func)
        list_netl.sort(key = sort_func)

        # append them all together into the new solution objects list
        solution_objects = list_2019 + list_2022 + list_princeton + list_netl + list_nrel

    # separate into cases
    plot_data[0]['case objects'], plot_data[1]['case objects'], plot_data[3]['case objects'], \
        plot_data[4]['case objects'], plot_data[5]['case objects'], plot_data[6]['case objects'] = separate_cases(solution_objects)

    if sort != 'source':
        # use python sort function to sort each of the case objects lists 
            for d in plot_data.keys():

                plot_data[d]['case objects'].sort(key = sort_func)

    else:
        for d in plot_data.keys():
            
            # perform sort by source 

            # separate list into years and different projections 
            list_2019 = []
            list_2022 = []
            list_nrel = []
            list_princeton = []
            list_netl = []

            for obj in plot_data[d]['case objects']:
                if obj.signal_label[-11:-7] == '2019':
                    list_2019.append(obj)
                elif obj.signal_label[-11:-7] == '2022':
                        list_2022.append(obj)

                elif obj.signal_label[-11:-7] == '2030':
                    list_princeton.append(obj)
                elif obj.signal_label[-11:-7] == '2035':
                    # check nrel 
                    if obj.signal_label[0:4] == 'MiNg':
                        list_nrel.append(obj)
                else:
                    list_netl.append(obj)

            # sort each of these lists by median lmp 
            sort_func = lambda soln_obj: np.median(soln_obj.lmp)

            list_2019.sort(key = sort_func)
            list_2022.sort(key = sort_func)
            list_nrel.sort(key = sort_func)
            list_princeton.sort(key = sort_func)
            list_netl.sort(key = sort_func)

            # append them all together into the new solution objects list
            plot_data[d]['case objects'] = list_2019 + list_2022 + list_princeton + list_netl + list_nrel

    # save the profits, capacity factors and outputs
    for key in plot_data.keys():
        for i in range(len(plot_data[key]['case objects'])):

            if PROFIT:
                if relative_case is not None:
                    # profit list
                    plot_data[key]['profit'].append(((plot_data[relative_case]['case objects'][i].profit - plot_data[key]['case objects'][i].profit)/plot_data[relative_case]['case objects'][i].profit)*100)
                else:
                    # profit_list 
                    plot_data[key]['profit'].append(plot_data[key]['case objects'][i].profit)

            if CAPACITY:
                # capacity_lists 
                if key == 6:
                    plot_data[key]['p capacity'].append(0)
                else:
                    plot_data[key]['p capacity'].append(sum(plot_data[key]['case objects'][i].p_dispatch)/(len(plot_data[key]['case objects'][i].horizon)*plot_data[key]['case objects'][i].plant_nameplate))

                if key == 0 or key == 1:
                    plot_data[key]['h capacity'].append(0)
                else:
                    plot_data[key]['h capacity'].append(sum(plot_data[key]['case objects'][i].h_dispatch)/(len(plot_data[key]['case objects'][i].horizon)*plot_data[key]['case objects'][i].h_nameplate))

            if OUTPUT:
                # output lists 
                plot_data[key]['p output'].append(sum(plot_data[key]['case objects'][i].p_dispatch))

                if key == 0 or key == 1:
                    plot_data[key]['h output'].append(0)
                else:
                    plot_data[key]['h output'].append(sum(plot_data[key]['case objects'][i].h_dispatch)*3600) 



    # in one list, remove the final 7 characters to create signal only labels (excluding _modelX)
    for i in range(len(plot_data[0]['case objects'])):
        # remove the end of the strings 
        plot_data[0]['case objects'][i].signal_label = plot_data[0]['case objects'][i].signal_label[:-7]

    final_labels = [plot_data[0]['case objects'][i].signal_label for i in range(len(plot_data[0]['case objects']))]

    # define formatting strings for each case
    fmt_str = {0: '.--', 1: 'o--', 3: 's-', 4: '^-', 5: 'P-', 6: 'D:'}
    colors = {0: 'r', 1 : 'b', 3 : 'g', 4 : 'm', 5 : 'darkorange', 6 : 'deeppink'}
    labels = {0: 'NGCC', 1 : 'SOFC', 3 : 'NGCC + SOEC', 4 : 'rSOC', 5 : 'SOFC + SOEC', 6 : 'SOEC'}
    if relative_case is not None:
        y_labels = {'profit': 'Optimal Annual Profit (M$/yr)',\
            'relative profit': 'Optimal Percent Change in Profit \nrelative to {0}'.format(labels[relative_case]),\
            'p capacity':'Annual Power Capacity Factor\n(MW Produced/MW Capacity)' ,\
            'h capacity': 'Annual Hydrogen Capacity Factor\n(kg Produced/kg Capacity)', \
            'p output': 'Annual Power Output (MW/yr)', \
            'h output': 'Annual Hydrogen Output (kg/yr)'}
    else:
        y_labels = {'profit': 'Optimal Annual Profit (M$/yr)',\
            'p capacity':'Annual Power Capacity Factor\n(MW Produced/MW Capacity)' ,\
            'h capacity': 'Annual Hydrogen Capacity Factor\n(kg Produced/kg Capacity)', \
            'p output': 'Annual Power Output (MW/yr)', \
            'h output': 'Annual Hydrogen Output (kg/yr)'}

    for key in plot_data[0]:

        # skip case objects
        if key == 'case objects':
            continue
        
        # make profit plot 
        fig, ax = plt.subplots(figsize = (20, 9))

        # create color map 
        cmap = plt.cm.get_cmap('gist_rainbow')

        if sort == 'median':
            # normalize it to be between the min and max median LMP 
            vmin = min([sort_func(c) for c in plot_data[0]['case objects']])
            vmax = max([sort_func(c) for c in plot_data[0]['case objects']])

            norm = Normalize(vmin, vmax)

            # set background color sections based on median lmp
            for c in range(len(plot_data[0]['case objects'])):
                ax.axvspan(c-0.4999, c+0.4999, ymin = 0.9, ymax = 0.99, alpha = 1.0, color = cmap(norm(sort_func(plot_data[0]['case objects'][c]))))
            

            # surround the outer box with a black border
            plt.axvline(x = 0-0.5, ymin = 0.9, ymax = 0.99, color = 'k', linewidth = 4)
            plt.axvline(x = len(final_labels)-1 + 0.5, ymin = 0.9, ymax = 0.99, color = 'k', linewidth = 4)


        # plot 0 line
        plt.axhline(y=0, color = 'black', linewidth = 3, alpha = 0.5)


        # plot data
        # print(final_labels)
        for c in plot_data.keys():
            ax.plot(range(len(final_labels)), plot_data[c][key], fmt_str[c], color = colors[c], label = labels[c], markersize = 8, linewidth = 3, zorder = 3)

        # add horizontal lines on colorbar    
        axis_len = ax.get_ylim()[1] - ax.get_ylim()[0]

        if sort == 'median':
            plt.axhline(y=ax.get_ylim()[1]-((1-0.9)*axis_len),xmin = 0.0095, xmax = 0.9901,  color = 'black', linewidth = 4, zorder = 1)
            plt.axhline(y=ax.get_ylim()[1]-((1-0.99)*axis_len),xmin = 0.0095, xmax = 0.9901,  color = 'black', linewidth = 4, zorder = 1)


            sortfuncvals = []
            multiples_of_10 = {10: 1000, 20:1000, 30:1000, 40:1000, 50:1000, 65: 1000}

            # label multiples of 10 on color bar 
            for c in range(len(plot_data[0]['case objects'])):
                # make a list of the values that sort func returns 
                sortfuncvals.append(sort_func(plot_data[0]['case objects'][c]))


            for k in multiples_of_10.keys():
                difference = 1000
                for j in range(len(sortfuncvals)):
                # find the closest median LMP to each multiple of 10 from 10 - 70  and save it in the dict 

                    if abs(sortfuncvals[j] - k) <= difference: 
                        multiples_of_10[k] = j
                        difference = abs(sortfuncvals[j] - k)

            for k in multiples_of_10.keys():
                # print the number over the bar at that value
                plt.text(x = multiples_of_10[k], y = ax.get_ylim()[1] - ((1-0.94)*axis_len), s = ('$' + str(k) + '/MWh'), color = 'k', fontweight = 'bold', fontsize = 14, zorder = 3)


        # remove margins
        plt.margins(x=0.01)

        # add hydrogen price label 
        plt.title('Hydrogen Price = ${0}/kg'.format(solution_objects[2].h2_price[0]), pad = 47, fontsize = 30, fontweight = 'bold')

        # set axes labels 
        if sort == 'median':
            plt.xlabel('Price Scenarios (sorted from lowest to highest median LMP)', fontsize = 30, fontweight = 'bold')
        elif sort == 'bimodal':
            plt.xlabel('Price Scenarios (sorted from lowest to highest bimodality)', fontsize = 30, fontweight = 'bold')
        elif sort == 'gasprice':
            plt.xlabel('Price Scenarios (sorted from lowest to highest natural gas price')
        elif sort == 'source':
            plt.xlabel('Price Scenarios (sorted chronologically and by projection source)', fontsize = 30, fontweight = 'bold')
        if relative_case is not None and key == 'profit':
            plt.ylabel(y_labels['relative profit'], fontsize = 30, fontweight = 'bold')
        else:
            plt.ylabel(y_labels[key], fontsize = 30, fontweight = 'bold')

        # major and minor ticks
        plt.tick_params(direction='in', top = True, right = True)
        plt.minorticks_on()
        plt.tick_params(which = 'minor', direction = 'in', top = False, bottom = False, right = True)


        # set y limits
        if relative_case is not None and key == 'profit':
            plt.ylim(ymin = -250, ymax = 250)

        # set labels
        plt.xticks(range(len(final_labels)), final_labels, size = 12,  fontweight = 'bold', rotation = 90)

        # add tick labels every 5 lines
        # calculate the length of the y axis 
        for i, label in enumerate(final_labels):
            if i % 5 == 0:
                if i == 0:
                    # label 1 not 0
                    plt.text(i, ax.get_ylim()[0]+(0.01*axis_len), i+1, ha = 'center', fontsize = 12, fontweight = 'bold')
                else:
                    plt.text(i-1, ax.get_ylim()[0]+(0.01*axis_len), i, ha='center', fontsize=12, fontweight = 'bold')

        plt.yticks(size = 20, fontweight = 'bold')

        # legend
        legend_properties = {'weight':'bold', 'size':20}
        plt.legend(loc = 'upper center', ncol = 6, bbox_to_anchor = (0.5, 1.1), prop = legend_properties)
        plt.grid(zorder = 2)

        

        # generate plot name 
        h2_price_string = str(solution_objects[3].h2_price[0])
        if relative_case is not None and key == 'profit':
            plot_name = 'all_{0}_{1}{2}_{3}_relativeto{4}'.format(key, h2_price_string.split('.')[0], h2_price_string.split('.')[1], sort, labels[relative_case])
        else:
            plot_name = 'all_{0}_{1}{2}_{3}'.format(key, h2_price_string.split('.')[0], h2_price_string.split('.')[1], sort)
        # print(plot_name)

        if save:
            plt.savefig((plot_name + '.png'), dpi = 300, bbox_inches = 'tight')
            plt.savefig((plot_name + '.pdf'), dpi = 300, bbox_inches = 'tight')

        plt.show()


    return

def plot_profit_scatter(fdest_data = 'formatted_raw_data.csv', x = 'log10p', y = 'medianlmp', h2_price = 20, side_by_side = False, save = False):
    '''
    Generates a scatter plot for a single case where marker size is determined by NPV.

    Arguments:
        fdest_data: string, destination of data file 
            default = 'formatted_raw_data.csv' data file in this directory
        x: string, what to plot on the x axes
            options = log10p, gasprice
        y: string, what to plot on the y axes
            options = medianlmp, gasprice
        h2_price: int, hydrogen price of data to be plotted. options = 0 (all prices), 10 ($1/kg), 15 ($1.5/kg), 20 ($2/kg), 25 ($2.5/kg), 30 ($3/kg)
            default = 20, $2/kg hydrogen data plotted
        side_by_side: boolean, if True, creates two subplots side by side that separate the negative profit and positive profit bubbles
            defualt = False, produces a single plot per case with all points regardless of NPV
        save: boolean, whether or not to save the plots
            default = False, do not save

    Returns:
        None
    '''

In [ ]:
def compute_computation_times(directory, save = False):
    '''
    Returns a table of minimum, average, and maximum computation times in minutes 
    as a data frame.

    Arguments:
        directory: Location of market results files 
            default = '../', folders are one directory up 
        save: boolean, whether or not to save the table as a csv
            default = False, do not save 

    Returns:
        comp_times: pd.DataFrame, minimum, average, and maximum compuitational times by system 
    '''

    # read all computational time file 
    time_10 = pd.read_csv((directory + '/market_results_10/00_time.csv'), header = None)
    time_15 = pd.read_csv((directory + '/market_results_15/00_time.csv'), header = None)
    time_20 = pd.read_csv((directory + '/market_results_20/00_time.csv'), header = None)
    time_25 = pd.read_csv((directory + '/market_results_25/00_time.csv'), header = None)
    time_30 = pd.read_csv((directory + '/market_results_30/00_time.csv'), header = None)  

    # create lists for each system
    NGCC = []
    SOFC = []
    NGCCSOEC = []
    rSOC = []
    SOFCSOEC = []
    SOEC = []

    # save time values from dataframe into lists
    for df in [time_10, time_15, time_20, time_25, time_30]:
        for i ,row in df.iterrows():
            if row[0] == 'model0':
                NGCC.append(row[4])

            elif row[0] == 'model1':
                SOFC.append(row[4])

            elif row[0] == 'model3':
                NGCCSOEC.append(row[4])

            elif row[0] == 'model4':
                rSOC.append(row[4])

            elif row[0] == 'model5':
                SOFCSOEC.append(row[4])

            elif row[0] == 'model6':
                SOEC.append(row[4])

            else:
                raise NotImplementedError()
            
    # make dataframe for the computation times 

    comp_times = pd.DataFrame(index = ['NGCC', 'SOFC', 'NGCC + SOEC', 'rSOC', 'SOFC + SOEC', 'SOEC'], columns = ['Minimum Solving Time (min)', 'Average Solving Time (min)', 'Maximum Solving Time (min)'])

    comp_times.at['NGCC', 'Minimum Solving Time (min)'] = round(np.min(NGCC),2)
    comp_times.at['NGCC', 'Average Solving Time (min)'] = round(np.mean(NGCC),2)
    comp_times.at['NGCC', 'Maximum Solving Time (min)'] = round(max(NGCC),2)

    comp_times.at['SOFC', 'Minimum Solving Time (min)'] = round(min(SOFC),2)
    comp_times.at['SOFC', 'Average Solving Time (min)'] = round(np.mean(SOFC),2)
    comp_times.at['SOFC', 'Maximum Solving Time (min)'] = round(max(SOFC),2)

    comp_times.at['NGCC + SOEC', 'Minimum Solving Time (min)'] = round(min(NGCCSOEC),2)
    comp_times.at['NGCC + SOEC', 'Average Solving Time (min)'] = round(np.mean(NGCCSOEC),2)
    comp_times.at['NGCC + SOEC', 'Maximum Solving Time (min)'] = round(max(NGCCSOEC),2)

    comp_times.at['rSOC', 'Minimum Solving Time (min)'] = round(min(rSOC),2)
    comp_times.at['rSOC', 'Average Solving Time (min)'] = round(np.mean(rSOC),2)
    comp_times.at['rSOC', 'Maximum Solving Time (min)'] = round(max(rSOC),2)

    comp_times.at['SOFC + SOEC', 'Minimum Solving Time (min)'] = round(min(SOFCSOEC),2)
    comp_times.at['SOFC + SOEC', 'Average Solving Time (min)'] = round(np.mean(SOFCSOEC),2)
    comp_times.at['SOFC + SOEC', 'Maximum Solving Time (min)'] = round(max(SOFCSOEC),2)

    comp_times.at['SOEC', 'Minimum Solving Time (min)'] = round(min(SOEC),2)
    comp_times.at['SOEC', 'Average Solving Time (min)'] = round(np.mean(SOEC),2)
    comp_times.at['SOEC', 'Maximum Solving Time (min)'] = round(max(SOEC),2)

    if save:
        comp_times.to_csv('comp_times.csv')

    return comp_times

def process_model_results(m_results_m):
    '''
    Process model results to calculate capacity factors and other metrics
    
    Arguments:
        m_results_m: dictionary of model results for different columns
        
    Returns:
        plot_data: dictionary containing processed data for plotting
    '''
    plot_data = {}
    
    for column_name, model in m_results_m.items():
        if model is None:
            continue
            
        # Initialize data structure for this column
        plot_data[column_name] = {
            'profit': 0,
            'p_capacity': 0,
            'p_output': 0
        }
        
        try:
            # Get power dispatch data
            power_dispatch = []
            for t in model.time:
                if hasattr(model, 'power_dispatch'):
                    power_dispatch.append(pyo.value(model.power_dispatch[t]))
                elif hasattr(model, 'fs') and hasattr(model.fs, 'power_dispatch'):
                    power_dispatch.append(pyo.value(model.fs.power_dispatch[t]))
            
            # Calculate metrics
            if power_dispatch:
                # Calculate total power output
                plot_data[column_name]['p_output'] = sum(power_dispatch)
                
                # Calculate capacity factor
                # Assuming nameplate capacity of 650 MW (from your model configuration)
                nameplate_capacity = 650
                time_periods = len(power_dispatch)
                plot_data[column_name]['p_capacity'] = sum(power_dispatch) / (time_periods * nameplate_capacity)
                
                # Get profit if available
                if hasattr(model, 'objective'):
                    plot_data[column_name]['profit'] = pyo.value(model.objective)
                elif hasattr(model, 'fs') and hasattr(model.fs, 'objective'):
                    plot_data[column_name]['profit'] = pyo.value(model.fs.objective)
                    
        except Exception as e:
            print(f"Error processing column {column_name}: {e}")
            continue
            
    return plot_data